1 - get the nodes state observations : their informations through the rounds + the accuracy of their models

In [181]:
import json

# Load the JSON data from the file
with open('../../subdir/results/scenario1/score_alpha02_nodes300/blockchain.json', 'r') as f:
    data = json.load(f)

# Create a list to store node information for each block
node_info_by_block = {}
transitions = []
nodes_by_block=[]
# transition = {"obs" : , "obs_" , "action" : , "reward" : ,"done" : }
# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']

    for transaction in json_obj['Transactions']:
        content = transaction.get('Content', {})
        message_type = content.get('message_type')
        node_id = content.get('nodeId')

        if message_type == 1 and node_id is not None:
            availability = int(content.get('availability'))
            honesty = content.get('honesty')
            dataset_size = content.get('datasetSize')
            freq = content.get('freq')
            trans_rate = content.get('transRate')
            task = content.get('task')
            dropout = content.get('dropout', 0.0)

            # Find accuracy for the corresponding node in message_type 2 transactions
            accuracy = 0.0
            for json_obj2 in data:
                if json_obj2['BlockId'] == block_id:
                    for transaction2 in json_obj2['Transactions']:
                        if transaction2['Content'].get('message_type') == 2 and transaction2['Content'].get('nodeId') == node_id and transaction2['Content'].get('type') == 0:
                            accuracy = transaction2['Content'].get('accuracy', 0.0)
                            break
                    if accuracy != 0.0:
                        break

            node_info = [node_id, availability, honesty, dataset_size, freq, trans_rate, dropout, accuracy]

            if block_id not in node_info_by_block:
                node_info_by_block[block_id] = []

            node_info_by_block[block_id].append(node_info)

# Sort the node lists within each block by nodeId
for nodes in node_info_by_block.values():
    nodes.sort(key=lambda x: x[0])  # Sort based on nodeId

# Print or use node_info_by_block as needed
for block_id, nodes in node_info_by_block.items():
    
   
        print(f"Block {block_id}: size {len(nodes)}")
        nodes_by_block.append(nodes)
        for node_info in nodes:  
            print(node_info)
            # if node_info[0] != nodes[node_info[0]][0]: print(node_info[0])
            

Block 1/0: size 300
[0, 1, 0.0, 860, 200.0, 410.0, 0.0, 0.0]
[1, 1, 0.0, 420, 280.0, 720.0, 0.0, 0.0]
[2, 1, 0.0, 890, 180.0, 270.0, 0.0, 0.0]
[3, 1, 0.0, 990, 220.0, 640.0, 0.0, 64.14]
[4, 0, 0.0, 460, 170.0, 170.0, 0.0, 0.0]
[5, 1, 0.0, 710, 150.0, 840.0, 0.0, 72.54]
[6, 0, 0.0, 410, 170.0, 920.0, 0.0, 0.0]
[7, 1, 0.0, 500, 270.0, 420.0, 0.0, 49.0]
[8, 1, 0.0, 360, 150.0, 640.0, 0.0, 56.94]
[9, 0, 0.0, 880, 290.0, 990.0, 0.0, 0.0]
[10, 1, 0.0, 320, 280.0, 630.0, 0.0, 0.0]
[11, 1, 0.0, 530, 170.0, 180.0, 0.0, 0.0]
[12, 1, 0.0, 370, 150.0, 980.0, 0.0, 0.0]
[13, 1, 0.0, 170, 160.0, 430.0, 0.0, 23.53]
[14, 1, 0.0, 960, 190.0, 450.0, 0.0, 0.0]
[15, 1, 0.0, 740, 130.0, 380.0, 0.0, 0.0]
[16, 1, 0.0, 290, 90.0, 850.0, 0.0, 56.9]
[17, 0, 0.0, 420, 250.0, 910.0, 0.0, 0.0]
[18, 0, 0.0, 100, 100.0, 220.0, 0.0, 0.0]
[19, 1, 0.0, 400, 60.0, 660.0, 0.0, 0.0]
[20, 1, 0.0, 870, 160.0, 770.0, 0.0, 67.24]
[21, 1, 0.0, 640, 300.0, 510.0, 0.0, 67.97]
[22, 1, 0.0, 900, 210.0, 830.0, 0.0, 67.22]
[23, 1, 0.

2 - get the global accuracy for each round

In [182]:
# Create a list to store the global accuracy values
global_accuracy = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 2 and type equal to 2
    accuracy_transaction = None
    for transaction in json_obj['Transactions']:
        content = transaction['Content']
        if content.get('message_type') == 2 and content.get('type') == 2:
            accuracy_transaction = transaction
            break
    
    if accuracy_transaction is not None:
        accuracy = accuracy_transaction['Content'].get('accuracy', 0.0)
        global_accuracy.append(accuracy)

# Print or use global_accuracy as needed
print("Global Accuracy Values:", global_accuracy)

Global Accuracy Values: [56.36, 77.27, 83.46, 78.46, 90.91, 91.67, 89.58, 89.62, 90.45, 90.0, 94.09, 90.88, 91.47, 95.45, 93.23, 94.19, 93.81, 93.64, 94.12, 90.77, 93.46, 95.93, 95.19, 96.67, 94.0, 95.33, 94.67, 97.06, 95.19]


In [183]:
# Create a list to store the concatenated lists for each block
selected_nodes = []

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes.append(aggregators + trainers)

# Print or use selected_nodes as needed
for block_id, selected_node_list in zip([json_obj['BlockId'] for json_obj in data], selected_nodes):
    print(f"Block {block_id}: {selected_node_list}")

Block 1/0: [70, 223, 256, 29, 63, 60, 67, 34, 264, 120, 185, 220, 51, 192, 172, 285, 265, 228, 65, 197, 114, 218, 293, 124, 10, 43, 1, 246, 119, 284, 225, 94, 177, 36, 12, 233, 204, 267, 164, 163, 131, 57, 196, 156, 59, 116, 227, 107, 182, 272, 209, 229, 260, 103, 263, 140, 113, 48, 13, 93, 8, 253, 215, 211, 254, 121, 66, 16, 83, 251, 274, 266, 21, 291, 87, 170, 178, 160, 7, 240, 136, 90, 45, 213, 82, 169, 31, 85, 138, 237, 179, 297, 139, 22, 143, 95, 25, 101, 189, 153, 41, 61, 117, 239, 248, 99, 280, 89, 286, 74, 5, 75, 123, 199, 166, 296, 161, 3, 137, 110, 122, 165, 37, 214, 88, 142, 298, 238, 20, 40]
Block 1/1: [223, 256, 29, 60, 67, 34, 264, 70, 120, 185, 220, 51, 192, 172, 285, 265, 228, 197, 65, 218, 124, 43, 1, 246, 69, 119, 284, 17, 63, 148, 225, 177, 36, 12, 233, 204, 267, 258, 164, 163, 57, 196, 156, 59, 116, 227, 107, 182, 81, 272, 35, 278, 260, 103, 111, 263, 140, 208, 113, 93, 290, 9, 217, 8, 253, 215, 211, 254, 121, 151, 155, 16, 251, 274, 266, 250, 21, 291, 273, 226, 170

In [184]:
round_reward = []
transitions = []

for block_id in range(0,29):  # Fixed the range
    # print(len(node_info_by_block) - 1)
    print(f"Block {block_id}:")
    nodes_rewards = []
    
    for node_index in selected_nodes[block_id]:
        # print("Node index: ", node_index)
                   
        node_data = nodes_by_block[block_id][node_index]
        reward = node_data[-1]
        # print("Original reward: ", reward)
        
        if reward != 0.0:
            reward *= node_data[2]
        else :
            reward = node_data[2]
        # print("Modified reward: ", reward)
        
        nodes_rewards.append(reward)
    
    # print("Nodes rewards:", nodes_rewards)
    cumulated_reward = sum(nodes_rewards)
    round_reward.append(cumulated_reward)

   
    transition = {
        "obs": nodes_by_block[block_id],
        "obs_": nodes_by_block[block_id + 1],
        "action": selected_nodes[block_id],
        "reward": cumulated_reward,
        "done": block_id == len(node_info_by_block) - 1  # Corrected the condition
    }
    transitions.append(transition)

for transition in transitions : 
    print(transition)


Block 0:
Block 1:
Block 2:
Block 3:
Block 4:
Block 5:
Block 6:
Block 7:
Block 8:
Block 9:
Block 10:
Block 11:
Block 12:
Block 13:
Block 14:
Block 15:
Block 16:
Block 17:
Block 18:
Block 19:
Block 20:
Block 21:
Block 22:
Block 23:
Block 24:
Block 25:
Block 26:
Block 27:
Block 28:
{'obs': [[0, 1, 0.0, 860, 200.0, 410.0, 0.0, 0.0], [1, 1, 0.0, 420, 280.0, 720.0, 0.0, 0.0], [2, 1, 0.0, 890, 180.0, 270.0, 0.0, 0.0], [3, 1, 0.0, 990, 220.0, 640.0, 0.0, 64.14], [4, 0, 0.0, 460, 170.0, 170.0, 0.0, 0.0], [5, 1, 0.0, 710, 150.0, 840.0, 0.0, 72.54], [6, 0, 0.0, 410, 170.0, 920.0, 0.0, 0.0], [7, 1, 0.0, 500, 270.0, 420.0, 0.0, 49.0], [8, 1, 0.0, 360, 150.0, 640.0, 0.0, 56.94], [9, 0, 0.0, 880, 290.0, 990.0, 0.0, 0.0], [10, 1, 0.0, 320, 280.0, 630.0, 0.0, 0.0], [11, 1, 0.0, 530, 170.0, 180.0, 0.0, 0.0], [12, 1, 0.0, 370, 150.0, 980.0, 0.0, 0.0], [13, 1, 0.0, 170, 160.0, 430.0, 0.0, 23.53], [14, 1, 0.0, 960, 190.0, 450.0, 0.0, 0.0], [15, 1, 0.0, 740, 130.0, 380.0, 0.0, 0.0], [16, 1, 0.0, 290, 90.0, 

In [187]:
import sys
sys.path.insert(0, '../')  # Add the directory containing "DRL" to the path

from agent import node_selection_agent as ag
from environment import node_selection_env as nds

total_nodes = 300
select_Agg = 50
select_trainer = 80
num_features = 8
target_accuracy = 99
max_rounds= 30
#create the nodes selection environment accoding to the observation
envNodeSelect = nds.FLNodeSelectionEnv(total_nodes,select_Agg,select_trainer, num_features=num_features, target=target_accuracy, max_rounds=max_rounds)
obs_shape = envNodeSelect.observation_space.sample()["current_state"].shape
print(obs_shape)
agent = ag.Agent(input_shape=obs_shape ,n_actions=obs_shape[0], env=envNodeSelect,max_actions=select_trainer+select_Agg) 
agent.load_models()

# train the agent with the transitions store the trasitions then make the agent learn
for transition in transitions : 
    agent.remember(transition["obs"],transition["action"], transition["reward"], transition["obs_"], transition["done"])
    agent.learn()

agent.save_models()


(300, 8)
.... loading models ....
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
...loading checkpoint...
.... finished loading models ....
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
updated the networks
.... saving models ....
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
...saving checkpoint...
upd

In [186]:
from collections import defaultdict


# Create a dictionary to store the concatenated lists for each block
selected_nodes = defaultdict(list)

# Iterate through the transactions of all blocks
for json_obj in data:
    block_id = json_obj['BlockId']
    
    # Find the transaction with message_type equal to 4
    aggregator_transaction = None
    for transaction in json_obj['Transactions']:
        if transaction['Content'].get('message_type') == 4:
            aggregator_transaction = transaction
            break
    
    if aggregator_transaction is not None:
        aggregators = aggregator_transaction['Content'].get('aggregators', [])
        trainers = aggregator_transaction['Content'].get('trainers', [])
        
        selected_nodes[block_id] = aggregators + trainers

# Count how many times each node was selected
node_selection_counts = defaultdict(int)
for selected_node_list in selected_nodes.values():
    for node_id in selected_node_list:
        node_selection_counts[node_id] += 1

# Print or use node_selection_counts as needed
for node_id, count in node_selection_counts.items():
    print(f"Node {node_id} was selected {count} times.")


Node 70 was selected 26 times.
Node 223 was selected 23 times.
Node 256 was selected 25 times.
Node 29 was selected 7 times.
Node 63 was selected 22 times.
Node 60 was selected 26 times.
Node 67 was selected 29 times.
Node 34 was selected 11 times.
Node 264 was selected 21 times.
Node 120 was selected 28 times.
Node 185 was selected 23 times.
Node 220 was selected 25 times.
Node 51 was selected 5 times.
Node 192 was selected 22 times.
Node 172 was selected 24 times.
Node 285 was selected 21 times.
Node 265 was selected 25 times.
Node 228 was selected 23 times.
Node 65 was selected 24 times.
Node 197 was selected 26 times.
Node 114 was selected 25 times.
Node 218 was selected 25 times.
Node 293 was selected 25 times.
Node 124 was selected 22 times.
Node 10 was selected 4 times.
Node 43 was selected 8 times.
Node 1 was selected 4 times.
Node 246 was selected 22 times.
Node 119 was selected 27 times.
Node 284 was selected 23 times.
Node 225 was selected 22 times.
Node 94 was selected 20 t